# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

Synthetic opioids effectively relieve acute and postoperative pain but are much less effective for chronic neuropathic pain. Long term use of opioids leads to serious side effects, such as increased pain sensitivity (hyperalgesia), physical dependence, and frequent progression to addiction, especially for chronic pain patients. The aim of this study is help manage withdrawal and pain in opioid-dependent patients.

The research found robust gene expression changes in the mouse reward system (nucleus accumbens and prefrontal cortex) after both nerve injury and drug interventions. identifying the epigenetic regulators HDAC1/HDAC2 as key target. Treatment with inhibitor RBC1HI showed improved withdrawal symptoms.

--> analyse transcriptomic effects

What do the conditions mean?

oxy: oxycodone (mice treated with it)


sal: saline solution (control group)

What do the genotypes mean?

SNI: Speared Nerve Injury (chronic pain)


Sham: control group, undergoes the same surgical procedures (such as anesthesia and incisions) as SNI, but without actual nerve damage or induction of chronic pain

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?
- Perform quality control on raw RNA-seq data (read quality, adapter trimming)
- Align reads to the mouse reference genome and quantify gene expression
- Normalize gene expression data to remove technical variability. E.g use quantile normalization, keep biological variation and exclude technical
- Exploratory analysis (PCA, clustering) to check for batch effects or outlier

Which groups would you compare to each other?
- Sham-Sal vs SNI-Sal: This comparison checks baseline molecular changes caused by neuropathic pain.
- Sham-Oxy vs Sham-Sal: Effect of oxycodone on gene expression without neuropathic pain, showing opioid effects in pain-free animals.
- SNI-Oxy vs SNI-Sal: Effect of oxycodone in animals with neuropathic pain, showing opioid effect plus interaction with chronic pain.
- SNI-Oxy vs Sham-Oxy: How chronic neuropathic pain modifies oxycodone response, isolating the interaction effect.

Please also mention which outcome you would expect to see from each comparison.

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [ ]:
import pandas as pd
data = pd.read_excel("conditions_runs_oxy_project.xlsx")


data.drop(columns=["Patient", "RNA-seq", "DNA-seq", "condition: Sal", "Genotype: Sham"], inplace=True)
# Convert "x" → 1, NaN → 0
df = data.fillna(0) #  -> not available
df = df.replace("x", 1) #  -> used 
df

/var/folders/tm/hjrdnkd97m7d0qsg6x7cv7300000gn/T/ipykernel_65305/189702353.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("x", 1) # df.replace("x", True) -> used


,Run,Condition: Oxy,Genotype: SNI
0,SRR23195505,0,1
1,SRR23195506,1,0
2,SRR23195507,0,0
3,SRR23195508,1,1
4,SRR23195509,1,1
5,SRR23195510,0,1
6,SRR23195511,1,0
7,SRR23195512,0,0
8,SRR23195513,0,1
9,SRR23195514,1,0


In [33]:
# Count 0 and 1 in each column (excluding "Run")
for col in df.columns[1:]:
    print(f"\nCounts for {col}:")
    print(df[col].value_counts())


Counts for Condition: Oxy:
Condition: Oxy
0    8
1    8
Name: count, dtype: int64

Counts for Genotype: SNI:
Genotype: SNI
1    8
0    8
Name: count, dtype: int64


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [ ]:
base_counts = pd.read_csv("base_counts.csv")
base_counts

full_data = pd.merge(df, base_counts)
full_data


In [45]:
full_data.sort_values(by ="Bases", ascending = 1)

,Run,Condition: Oxy,Genotype: SNI,Bases
11,SRR23195516,1,1,6203117700
6,SRR23195511,1,0,6456390900
12,SRR23195517,1,1,6863840400
0,SRR23195505,0,1,6922564500
3,SRR23195508,1,1,6927786900
14,SRR23195519,1,0,6996050100
4,SRR23195509,1,1,7003550100
9,SRR23195514,1,0,7226808600
5,SRR23195510,0,1,7377388500
7,SRR23195512,0,0,7462857900


While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.